In [283]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np

import libry as ry
import time
print(cv.__version__)

4.5.3


In [284]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/ex05_redball.g")
D = RealWorld.view()

ball = RealWorld.getFrame("ball")
ball.setContact(1)

In [285]:
#RealWorld.addFrame(camera_fixpoint)

In [286]:
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.bullet, verbose=1)
S.addSensor("camera")

In [287]:
# we're adding an "imp" to the simulation, which is a little process that can inject perturbations
S.addImp(ry.ImpType.objectImpulses, ['ball'], [])

In [288]:
# create your model world
C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
cameraFrame = C.frame("camera")
D2 = C.view()

#the focal length
f = 0.895
f = f * 360.
fxfypxpy = [f, f, 320., 180.]

In [289]:

config_obj_name = "conf_ball"

# delete if necceary
#C.delFrame(config_obj_name)

obj = C.addFrame(config_obj_name,"camera")

obj.setShape(ry.ST.sphere, [.03])
obj.setPosition([0., .1, 1.6])
obj.setColor([1.,0,0])


objectPosition = RealWorld.getFrame("ball").getPosition()

#set the model object to percept
obj.setPosition(objectPosition)



# Setup complete here!

## OpenCV find red ball code:

In [290]:
def _segment_redball(rgb):
    rgb = cv.cvtColor(rgb, cv.COLOR_BGR2RGB)
    hsv = cv.cvtColor(rgb, cv.COLOR_BGR2HSV)

    mask1 = cv.inRange(hsv, (0, 120, 70), (10, 255, 255))
    mask2 = cv.inRange(hsv, (170, 120, 70), (180, 255, 255))

    # if len(mask1)>0: cv.imshow('OPENCV - mask1',  mask1)
    # if len(mask2)>0: cv.imshow('OPENCV - mask2',  mask2)
    mask = mask1 + mask2

    # create empty mask
    filtered_mask = np.zeros(mask.shape, np.uint8)

    # if len(filtered_mask)>0: cv.imshow('OPENCV - filtered_mask',  filtered_mask)
    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        # find largest contour
        largest, idx = 0, None
        for i, c in enumerate(contours):
            # remove noise
            if c.shape[0] < 10: continue
            if cv.contourArea(c) > largest:
                largest = cv.contourArea(c)
                idx = i
        if i is not None: 
            idx=0
        cv.drawContours(filtered_mask, contours[idx], -1, (255, 255, 255), -1)
    return filtered_mask


def _image_pointcloud(depth, mask):
    # if len(mask) > 0: cv.imshow('OPENCV - mask1', mask * 255)
    # cv.waitKey(100)
    mask_pixels = np.where(mask > 0)
    pointcloud = np.empty((mask_pixels[0].shape[0], 3))
    pointcloud[:, 0] = mask_pixels[1]  # x pixels
    pointcloud[:, 1] = mask_pixels[0]  # y pixels
    pointcloud[:, 2] = depth[mask_pixels[0], mask_pixels[1]]
    return pointcloud


def _meter_pointcloud(pixel_points, fxfypxpy):
    points = np.empty(np.shape(pixel_points))
    for i, p in enumerate(pixel_points):
        x = p[0]
        y = p[1]
        d = p[2]

        px = fxfypxpy[-2]
        py = fxfypxpy[-1]

        x_ = d * (x - px) / fxfypxpy[0]
        y_ = -d * (y - py) / fxfypxpy[1]
        z_ = -d
        points[i] = [x_, y_, z_]
    return points


def find_ball(rgb, depth, fxfypxpy):
    mask = _segment_redball(rgb)
    pixel_points = _image_pointcloud(depth, mask)
    obj_points = _meter_pointcloud(pixel_points, fxfypxpy)
    return obj_points, mask

# Lift and drop ball code:

In [291]:
def liftAndDrop(C, gripper, lift_time = 0.7, steps = 10):
    # calculate a lift of the ball
    
    # we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
    komo = C.komo_path(lift_time,steps, 3., True)
    komo.addObjective(  [lift_time], 
                        ry.FS.position, 
                        [gripper], 
                        ry.OT.sos, 
                        [1e2],
                        [.2,.2,1.2])
    komo.addObjective(  [lift_time], 
                        ry.FS.qItself, 
                        [], 
                        ry.OT.eq, 
                        [1e2], 
                        order=1)
    komo.addObjective(  [lift_time], 
                        ry.FS.vectorY, 
                        [gripper], 
                        ry.OT.sos, 
                        [1e2],
                        [0.,0.,1]);
    komo.addObjective(  [lift_time], 
                        ry.FS.vectorX, 
                        [gripper], 
                        ry.OT.sos, 
                        [1e2],
                        [0.,1.,0.]);
    komo.optimize()

    # here loop for upwards move
    for frame,tau in zip(komo.getPathFrames(),komo.getPathTau()):
        time.sleep(tau)
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

        C.setFrameState(frame)

        q= C.getJointState()

        #send position to the simulation
        S.step(q, tau, ry.ControlMode.position)

    tau = 0.01
    for t in range(100):
        time.sleep(tau)
        
        if t%20 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

        if S.getGripperWidth(gripper) < 0.04:
            S.openGripper(gripper,speed=2.)

        if(S.getGripperWidth(gripper)>0.04 and not S.getGripperIsGrasping(gripper)):
            print("DOPPED!")
            break
        

        S.step([], tau, ry.ControlMode.none)




In [292]:
def pickBall(C,gripper):
    points = []
    tau = .01

    gripping = False
    grasped = False
    
    objectPosition = [0., 0., 0.]

    for t in range(400):
        time.sleep(0.01)
        #grab sensor readings from the simulation
        q = S.get_q()
        if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
            points = S.depthData2pointCloud(depth, fxfypxpy)
            cameraFrame.setPointCloud(points, rgb)

            pos_bkp = objectPosition
            # get points of ball with opencv
            obj_points, mask = find_ball(rgb,depth,fxfypxpy)

            # get position of ball with mean of points
            objectPosition = obj_points.mean(axis=0)

            if np.isnan(objectPosition).any():
                objectPosition = pos_bkp

            #set the model object to percept (relative to camera because the position from the pointcloud is in camera coord.)
            obj.setRelativePosition(objectPosition)

        C.setJointState(q) #set your robot model to match the real q

        y,_ = C.evalFeature(ry.FS.positionDiff, [gripper, config_obj_name])
        distance = np.linalg.norm(y)
        # start grapsing here
    
        if t > 40 and not gripping:
            # we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
            komo = C.komo_path(1.,10, 3., True)
            komo.add_qControlObjective([], 1, 1.)
            komo.addObjective([1.], 
                              ry.FS.positionDiff, 
                              [gripper, config_obj_name], 
                              ry.OT.eq, 
                              [1e2],
                              [0.,-0.,0.]);
            komo.addObjective([1.], 
                              ry.FS.distance, 
                              [gripper, config_obj_name], 
                              ry.OT.sos, 
                               [1e2],
                              [-.06]);
            komo.addObjective([1.], 
                              ry.FS.vectorZ, 
                              [gripper], 
                              ry.OT.sos, 
                              [1e2],
                              [1.7,0.,1]);
            komo.addObjective([1.], 
                              ry.FS.vectorX, 
                              [gripper], 
                              ry.OT.sos, 
                              [1e2],
                              [0.,1.,0.]);

            # optimize
            komo.optimize()
            
            if distance <.2:
                C.setFrameState(komo.getFrameState(max(9,int(9-distance*5*7))))
            else:
                C.setFrameState(komo.getFrameState(2))
            # get joint states
            q = C.getJointState()


        
        if not gripping and (distance < .02):
            S.closeGripper(gripper, speed = 2.)
            gripping = True
        
        
        if gripping and (S.getGripperWidth(gripper) < .01):
            gripping=False
            S.openGripper(gripper, speed = 2.)

        if gripping  and S.getGripperIsGrasping(gripper):
            print("GRASPED!")
            grasped = True
            break

        #send no controls to the simulation
        S.step(q, tau, ry.ControlMode.position)

        
        




# Simulation start here

In [293]:
tau = .01



for t in range(70):
    time.sleep(0.01)
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        cameraFrame.setPointCloud(points, rgb)
    S.step([], tau, ry.ControlMode.none)
        


In [294]:
pickBall(C,"R_gripper")

** KOMO::run solver:sparse collisions:1 x-dim:140 T:10 k:2 phases:1 stepsPerPhase:10 tau:0.3  #timeSlices:12 #totalDOFs:140 #frames:1176
** optimization time:0.113386 (kin:0.00195 coll:0.081625 feat:0.017224 newton: 0.004654) setJointStateCount:34
   sos:9536.56 ineq:0 eq:0.080755
** KOMO::run solver:sparse collisions:1 x-dim:140 T:10 k:2 phases:1 stepsPerPhase:10 tau:0.3  #timeSlices:12 #totalDOFs:140 #frames:1176
** optimization time:0.088755 (kin:0.001504 coll:0.058074 feat:0.013456 newton: 0.009165) setJointStateCount:28
   sos:9535.79 ineq:0 eq:0.0736012
** KOMO::run solver:sparse collisions:1 x-dim:140 T:10 k:2 phases:1 stepsPerPhase:10 tau:0.3  #timeSlices:12 #totalDOFs:140 #frames:1176
** optimization time:0.074299 (kin:0.001896 coll:0.052949 feat:0.010637 newton: 0.00318) setJointStateCount:25
   sos:9535.26 ineq:0 eq:0.0696469
** KOMO::run solver:sparse collisions:1 x-dim:140 T:10 k:2 phases:1 stepsPerPhase:10 tau:0.3  #timeSlices:12 #totalDOFs:140 #frames:1176
** optimizatio

In [295]:
liftAndDrop(C,"R_gripper")

** KOMO::run solver:sparse collisions:1 x-dim:98 T:7 k:2 phases:0.7 stepsPerPhase:10 tau:0.3  #timeSlices:9 #totalDOFs:98 #frames:882
** optimization time:0.021257 (kin:0.000442 coll:0.015533 feat:0.002172 newton: 0.001032) setJointStateCount:10
   sos:1.86125 ineq:0 eq:8.12612e-06
DOPPED!


In [296]:
for round in range (5):
    tau = .01

    for t in range(100):
        time.sleep(0.01)
        if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
            points = S.depthData2pointCloud(depth, fxfypxpy)
            cameraFrame.setPointCloud(points, rgb)
        S.step([], tau, ry.ControlMode.none)
    
    pickBall(C,"R_gripper")
    
    liftAndDrop(C,"R_gripper")


** KOMO::run solver:sparse collisions:1 x-dim:140 T:10 k:2 phases:1 stepsPerPhase:10 tau:0.3  #timeSlices:12 #totalDOFs:140 #frames:1176
** optimization time:0.066361 (kin:0.001093 coll:0.050771 feat:0.007206 newton: 0.002649) setJointStateCount:24
   sos:9535.8 ineq:0 eq:0.0187545
** KOMO::run solver:sparse collisions:1 x-dim:140 T:10 k:2 phases:1 stepsPerPhase:10 tau:0.3  #timeSlices:12 #totalDOFs:140 #frames:1176
** optimization time:0.064723 (kin:0.001122 coll:0.048409 feat:0.007766 newton: 0.002723) setJointStateCount:24
   sos:9535.9 ineq:0 eq:0.0141649
** KOMO::run solver:sparse collisions:1 x-dim:140 T:10 k:2 phases:1 stepsPerPhase:10 tau:0.3  #timeSlices:12 #totalDOFs:140 #frames:1176
** optimization time:0.071393 (kin:0.001274 coll:0.045255 feat:0.014318 newton: 0.005123) setJointStateCount:27
   sos:9535.16 ineq:0 eq:0.0180694
** KOMO::run solver:sparse collisions:1 x-dim:140 T:10 k:2 phases:1 stepsPerPhase:10 tau:0.3  #timeSlices:12 #totalDOFs:140 #frames:1176
** optimizati

In [297]:
cv.destroyAllWindows()